In [1]:
import cv2
import numpy as np
import os
import subprocess

def iterative_superglue_stitch(img1_name, img2_name, input_dir, output_dir, temp_txt='temp_pairs.txt'):
    def load_image(name): return cv2.imread(os.path.join(input_dir, name))

    def generate_pair_txt(file_path, img1, img2):
        with open(file_path, 'w') as f:
            f.write(f"{img1} {img2}\n")

    def run_superglue(txt_filename):
        command = [
            "python", "match_pairs.py",
            "--resize", "-1",
            "--superglue", "outdoor",
            "--max_keypoints", "2048",
            "--nms_radius", "5",
            "--resize_float",
            "--input_dir", input_dir,
            "--input_pairs", txt_filename,
            "--output_dir", output_dir,
            "--viz",
            "--keypoint_threshold", "0.05",
            "--match_threshold", "0.9"
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        if result.returncode != 0:
            print("SuperGlue failed!", result.stderr)
            return False
        return True

    def load_npz_matches(file_name):
        npz = np.load(os.path.join(output_dir, file_name))
        matches = npz['matches']
        valid = matches > -1
        kp1 = npz['keypoints0'][valid]
        kp2 = npz['keypoints1'][matches[valid]]
        return kp1, kp2

    def crop_black(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(thresh)
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]

    def stitch_from_numpy_matches(img1, img2, kp1, kp2):
        pts1 = np.float32(kp1).reshape(-1, 1, 2)
        pts2 = np.float32(kp2).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)
        if H is None:
            raise ValueError("Homography failed.")
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
        warped_corners = cv2.perspectiveTransform(corners, H)
        all_corners = np.concatenate((np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2), warped_corners), axis=0)
        [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
        translation = [-xmin, -ymin]
        H_translation = np.array([[1, 0, translation[0]],
                                  [0, 1, translation[1]],
                                  [0, 0, 1]])
        result = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
        result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]] = img1
        return crop_black(result)

    # Run SuperGlue matching
    generate_pair_txt(temp_txt, img1_name, img2_name)
    match_file = f"{img1_name[:-4]}_{img2_name[:-4]}_matches.npz"
    if not os.path.exists(os.path.join(output_dir, match_file)):
        success = run_superglue(temp_txt)
        if not success:
            raise RuntimeError(f"SuperGlue matching failed between {img1_name} and {img2_name}")

    # Load matches and images
    kp1, kp2 = load_npz_matches(match_file)
    img1 = load_image(img1_name)
    img2 = load_image(img2_name)

    stitched_img = stitch_from_numpy_matches(img1, img2, kp1, kp2)
    stitched_name = f"{img1_name[:-4]}_{img2_name[:-4]}.jpg"
    cv2.imwrite(os.path.join(input_dir, stitched_name), stitched_img)
    return stitched_name


def hierarchical_overlap_stitching(start_idx, end_idx, input_dir, output_dir):
    """
    Iteratively stitch overlapping adjacent image chains:
    Round 0: 1_2, 2_3, 3_4, ...
    Round 1: 1_2_3 (from 1_2 and 2_3), 2_3_4 (from 2_3 and 3_4), ...
    Repeat until 1_2_3_..._n is produced.
    """
    # Initialize with individual images
    current_images = [f"{i}.jpg" for i in range(start_idx, end_idx + 1)]
    round_num = 0

    while len(current_images) > 1:
        print(f"\n--- Stitching Round {round_num+1} ---")
        new_images = []
        for i in range(len(current_images) - 1):
            img1 = current_images[i]
            img2 = current_images[i+1]
            print(f"Stitching {img1} + {img2}")
            try:
                stitched = iterative_superglue_stitch(img1, img2, input_dir, output_dir)
                new_images.append(stitched)
            except Exception as e:
                print(f"❌ Skipping pair {img1} + {img2} due to error: {e}")

        # Update current_images for next round
        current_images = new_images
        round_num += 1

    print(f"\n✅ Final stitched image: {current_images[0]}")
    return current_images[0]


# Call the new hierarchical overlapping stitching method
hierarchical_overlap_stitching(1, 48, input_dir="cropped", output_dir="car_panorama2/output/")



--- Stitching Round 1 ---
Stitching 1.jpg + 2.jpg
Stitching 2.jpg + 3.jpg
Stitching 3.jpg + 4.jpg
Stitching 4.jpg + 5.jpg
Stitching 5.jpg + 6.jpg
Stitching 6.jpg + 7.jpg
Stitching 7.jpg + 8.jpg
Stitching 8.jpg + 9.jpg
Stitching 9.jpg + 10.jpg
Stitching 10.jpg + 11.jpg
Stitching 11.jpg + 12.jpg
Stitching 12.jpg + 13.jpg
Stitching 13.jpg + 14.jpg
Stitching 14.jpg + 15.jpg
Stitching 15.jpg + 16.jpg
Stitching 16.jpg + 17.jpg
Stitching 17.jpg + 18.jpg
Stitching 18.jpg + 19.jpg
Stitching 19.jpg + 20.jpg
Stitching 20.jpg + 21.jpg
Stitching 21.jpg + 22.jpg
Stitching 22.jpg + 23.jpg
Stitching 23.jpg + 24.jpg
Stitching 24.jpg + 25.jpg
Stitching 25.jpg + 26.jpg
Stitching 26.jpg + 27.jpg
Stitching 27.jpg + 28.jpg
Stitching 28.jpg + 29.jpg
Stitching 29.jpg + 30.jpg
Stitching 30.jpg + 31.jpg
Stitching 31.jpg + 32.jpg
Stitching 32.jpg + 33.jpg
Stitching 33.jpg + 34.jpg
Stitching 34.jpg + 35.jpg
Stitching 35.jpg + 36.jpg
Stitching 36.jpg + 37.jpg
Stitching 37.jpg + 38.jpg
Stitching 38.jpg + 39.jpg
St

IndexError: list index out of range

In [2]:
import cv2
import numpy as np
import os
import subprocess

def iterative_superglue_stitch(img1_name, img2_name, input_dir, output_dir, stitched_name=None, temp_txt='temp_pairs.txt'):
    def load_image(name): return cv2.imread(os.path.join(input_dir, name))

    def generate_pair_txt(file_path, img1, img2):
        with open(file_path, 'w') as f:
            f.write(f"{img1} {img2}\n")

    def run_superglue(txt_filename):
        command = [
            "python", "match_pairs.py",
            "--resize", "-1",
            "--superglue", "outdoor",
            "--max_keypoints", "2048",
            "--nms_radius", "5",
            "--resize_float",
            "--input_dir", input_dir,
            "--input_pairs", txt_filename,
            "--output_dir", output_dir,
            "--viz",
            "--keypoint_threshold", "0.05",
            "--match_threshold", "0.9"
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        if result.returncode != 0:
            print("SuperGlue failed!", result.stderr)
            return False
        return True

    def load_npz_matches(file_name):
        npz = np.load(os.path.join(output_dir, file_name))
        matches = npz['matches']
        valid = matches > -1
        kp1 = npz['keypoints0'][valid]
        kp2 = npz['keypoints1'][matches[valid]]
        return kp1, kp2

    def crop_black(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(thresh)
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]

    def stitch_from_numpy_matches(img1, img2, kp1, kp2):
        pts1 = np.float32(kp1).reshape(-1, 1, 2)
        pts2 = np.float32(kp2).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)
        if H is None:
            raise ValueError("Homography failed.")
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
        warped_corners = cv2.perspectiveTransform(corners, H)
        all_corners = np.concatenate((np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2), warped_corners), axis=0)
        [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
        translation = [-xmin, -ymin]
        H_translation = np.array([[1, 0, translation[0]],
                                  [0, 1, translation[1]],
                                  [0, 0, 1]])
        result = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
        result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]] = img1
        return crop_black(result)

    # Generate .npz filename safely
    img1_base = os.path.splitext(os.path.basename(img1_name))[0]
    img2_base = os.path.splitext(os.path.basename(img2_name))[0]
    match_file = f"{img1_base}_{img2_base}_matches.npz"

    # Run SuperGlue matching
    generate_pair_txt(temp_txt, img1_name, img2_name)
    if not os.path.exists(os.path.join(output_dir, match_file)):
        success = run_superglue(temp_txt)
        if not success:
            raise RuntimeError(f"SuperGlue matching failed between {img1_name} and {img2_name}")

    # Load matches and images
    kp1, kp2 = load_npz_matches(match_file)
    img1 = load_image(img1_name)
    img2 = load_image(img2_name)

    stitched_img = stitch_from_numpy_matches(img1, img2, kp1, kp2)

    if stitched_name is None:
        stitched_name = f"{img1_base}_{img2_base}.jpg"

    stitched_path = os.path.join(input_dir, stitched_name)
    cv2.imwrite(stitched_path, stitched_img)
    return stitched_name


def hierarchical_overlap_stitching(start_idx, end_idx, input_dir, output_dir):
    """
    Iteratively stitch overlapping adjacent image chains:
    Round 0: 1_2, 2_3, 3_4, ...
    Round 1: 1_2_3 (from 1_2 and 2_3), 2_3_4 (from 2_3 and 3_4), ...
    Repeat until 1_2_3_..._n is produced.
    """
    current_images = [f"{i}.jpg" for i in range(start_idx, end_idx + 1)]
    round_num = 0

    while len(current_images) > 1:
        print(f"\n--- Stitching Round {round_num + 1} ---")
        new_images = []
        for i in range(len(current_images) - 1):
            img1 = current_images[i]
            img2 = current_images[i + 1]
            print(f"Stitching {img1} + {img2}")
            try:
                stitched_name = f"stitched_r{round_num}_i{i}.jpg"
                stitched = iterative_superglue_stitch(img1, img2, input_dir, output_dir, stitched_name)
                new_images.append(stitched)
            except Exception as e:
                print(f"❌ Skipping pair {img1} + {img2} due to error: {e}")

        if not new_images:
            raise RuntimeError("All stitching attempts failed. No images to proceed with.")

        current_images = new_images
        round_num += 1

    print(f"\n✅ Final stitched image: {current_images[0]}")
    return current_images[0]


# Call it like this:
hierarchical_overlap_stitching(1, 48, input_dir="cropped", output_dir="car_panorama2/output/")



--- Stitching Round 1 ---
Stitching 1.jpg + 2.jpg
Stitching 2.jpg + 3.jpg
Stitching 3.jpg + 4.jpg
Stitching 4.jpg + 5.jpg
Stitching 5.jpg + 6.jpg
Stitching 6.jpg + 7.jpg
Stitching 7.jpg + 8.jpg
Stitching 8.jpg + 9.jpg
Stitching 9.jpg + 10.jpg
Stitching 10.jpg + 11.jpg
Stitching 11.jpg + 12.jpg
Stitching 12.jpg + 13.jpg
Stitching 13.jpg + 14.jpg
Stitching 14.jpg + 15.jpg
Stitching 15.jpg + 16.jpg
Stitching 16.jpg + 17.jpg
Stitching 17.jpg + 18.jpg
Stitching 18.jpg + 19.jpg
Stitching 19.jpg + 20.jpg
Stitching 20.jpg + 21.jpg
Stitching 21.jpg + 22.jpg
Stitching 22.jpg + 23.jpg
Stitching 23.jpg + 24.jpg
Stitching 24.jpg + 25.jpg
Stitching 25.jpg + 26.jpg
Stitching 26.jpg + 27.jpg
Stitching 27.jpg + 28.jpg
Stitching 28.jpg + 29.jpg
Stitching 29.jpg + 30.jpg
Stitching 30.jpg + 31.jpg
Stitching 31.jpg + 32.jpg
Stitching 32.jpg + 33.jpg
Stitching 33.jpg + 34.jpg
Stitching 34.jpg + 35.jpg
Stitching 35.jpg + 36.jpg
Stitching 36.jpg + 37.jpg
Stitching 37.jpg + 38.jpg
Stitching 38.jpg + 39.jpg
St

'stitched_r46_i0.jpg'

In [3]:
import cv2
import numpy as np
import os
import subprocess

def iterative_superglue_stitch(img1_name, img2_name, input_dir, output_dir, stitched_name=None, temp_txt='temp_pairs.txt'):
    def load_image(name): return cv2.imread(os.path.join(input_dir, name))

    def generate_pair_txt(file_path, img1, img2):
        with open(file_path, 'w') as f:
            f.write(f"{img1} {img2}\n")

    def run_superglue(txt_filename):
        command = [
            "python", "match_pairs.py",
            "--resize", "-1",
            "--superglue", "outdoor",
            "--max_keypoints", "2048",
            "--nms_radius", "5",
            "--resize_float",
            "--input_dir", input_dir,
            "--input_pairs", txt_filename,
            "--output_dir", output_dir,
            "--viz",
            "--keypoint_threshold", "0.05",
            "--match_threshold", "0.9"
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        if result.returncode != 0:
            print("SuperGlue failed!", result.stderr)
            return False
        return True

    def load_npz_matches(file_name):
        npz = np.load(os.path.join(output_dir, file_name))
        matches = npz['matches']
        valid = matches > -1
        kp1 = npz['keypoints0'][valid]
        kp2 = npz['keypoints1'][matches[valid]]
        return kp1, kp2

    def crop_black(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(thresh)
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]

    def stitch_from_numpy_matches(img1, img2, kp1, kp2):
        pts1 = np.float32(kp1).reshape(-1, 1, 2)
        pts2 = np.float32(kp2).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)
        if H is None:
            raise ValueError("Homography failed.")
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
        warped_corners = cv2.perspectiveTransform(corners, H)
        all_corners = np.concatenate((np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2), warped_corners), axis=0)
        [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
        translation = [-xmin, -ymin]
        H_translation = np.array([[1, 0, translation[0]],
                                  [0, 1, translation[1]],
                                  [0, 0, 1]])
        result = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
        result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]] = img1
        return crop_black(result)

    # Generate .npz filename safely
    img1_base = os.path.splitext(os.path.basename(img1_name))[0]
    img2_base = os.path.splitext(os.path.basename(img2_name))[0]
    match_file = f"{img1_base}_{img2_base}_matches.npz"

    # Run SuperGlue matching
    generate_pair_txt(temp_txt, img1_name, img2_name)
    if not os.path.exists(os.path.join(output_dir, match_file)):
        success = run_superglue(temp_txt)
        if not success:
            raise RuntimeError(f"SuperGlue matching failed between {img1_name} and {img2_name}")

    # Load matches and images
    kp1, kp2 = load_npz_matches(match_file)
    img1 = load_image(img1_name)
    img2 = load_image(img2_name)

    stitched_img = stitch_from_numpy_matches(img1, img2, kp1, kp2)

    if stitched_name is None:
        stitched_name = f"{img1_base}_{img2_base}.jpg"

    stitched_path = os.path.join(input_dir, stitched_name)
    cv2.imwrite(stitched_path, stitched_img)
    return stitched_name


def hierarchical_overlap_stitching(start_idx, end_idx, input_dir, output_dir):
    """
    Iteratively stitch overlapping adjacent image chains:
    Round 0: 1_2, 2_3, 3_4, ...
    Round 1: 1_2_3 (from 1_2 and 2_3), 2_3_4 (from 2_3 and 3_4), ...
    Repeat until 1_2_3_..._n is produced.
    """
    current_images = [f"{i}.jpg" for i in range(start_idx, end_idx + 1)]
    round_num = 0

    while len(current_images) > 1:
        print(f"\n--- Stitching Round {round_num + 1} ---")
        new_images = []
        for i in range(len(current_images) - 1):
            img1 = current_images[i]
            img2 = current_images[i + 1]
            print(f"Stitching {img1} + {img2}")
            try:
                stitched_name = f"stitched_r{round_num}_i{i}.jpg"
                stitched = iterative_superglue_stitch(img1, img2, input_dir, output_dir, stitched_name)
                new_images.append(stitched)
            except Exception as e:
                print(f"❌ Skipping pair {img1} + {img2} due to error: {e}")

        if not new_images:
            raise RuntimeError("All stitching attempts failed. No images to proceed with.")

        current_images = new_images
        round_num += 1

    print(f"\n✅ Final stitched image: {current_images[0]}")
    return current_images[0]


# Call it like this:
hierarchical_overlap_stitching(1, 48, input_dir="Set-01", output_dir="car_panorama2/output/")



--- Stitching Round 1 ---
Stitching 1.jpg + 2.jpg
Stitching 2.jpg + 3.jpg
Stitching 3.jpg + 4.jpg
Stitching 4.jpg + 5.jpg
Stitching 5.jpg + 6.jpg
Stitching 6.jpg + 7.jpg
Stitching 7.jpg + 8.jpg
Stitching 8.jpg + 9.jpg
Stitching 9.jpg + 10.jpg
Stitching 10.jpg + 11.jpg
Stitching 11.jpg + 12.jpg
Stitching 12.jpg + 13.jpg
Stitching 13.jpg + 14.jpg
Stitching 14.jpg + 15.jpg
Stitching 15.jpg + 16.jpg
Stitching 16.jpg + 17.jpg
Stitching 17.jpg + 18.jpg
Stitching 18.jpg + 19.jpg
Stitching 19.jpg + 20.jpg
Stitching 20.jpg + 21.jpg
Stitching 21.jpg + 22.jpg
Stitching 22.jpg + 23.jpg
Stitching 23.jpg + 24.jpg
Stitching 24.jpg + 25.jpg
Stitching 25.jpg + 26.jpg
Stitching 26.jpg + 27.jpg
Stitching 27.jpg + 28.jpg
Stitching 28.jpg + 29.jpg
Stitching 29.jpg + 30.jpg
Stitching 30.jpg + 31.jpg
Stitching 31.jpg + 32.jpg
Stitching 32.jpg + 33.jpg
Stitching 33.jpg + 34.jpg
Stitching 34.jpg + 35.jpg
Stitching 35.jpg + 36.jpg
Stitching 36.jpg + 37.jpg
Stitching 37.jpg + 38.jpg
Stitching 38.jpg + 39.jpg
St

'stitched_r46_i0.jpg'

In [4]:
import cv2
import numpy as np
import os
import subprocess

def iterative_superglue_stitch(img1_name, img2_name, input_dir, output_dir, stitched_name=None, temp_txt='temp_pairs.txt'):
    def load_image(name): return cv2.imread(os.path.join(input_dir, name))

    def generate_pair_txt(file_path, img1, img2):
        with open(file_path, 'w') as f:
            f.write(f"{img1} {img2}\n")

    def run_superglue(txt_filename):
        command = [
            "python", "match_pairs.py",
            "--resize", "-1",
            "--superglue", "outdoor",
            "--max_keypoints", "2048",
            "--nms_radius", "5",
            "--resize_float",
            "--input_dir", input_dir,
            "--input_pairs", txt_filename,
            "--output_dir", output_dir,
            "--viz",
            "--keypoint_threshold", "0.05",
            "--match_threshold", "0.9"
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        if result.returncode != 0:
            print("SuperGlue failed!", result.stderr)
            return False
        return True

    def load_npz_matches(file_name):
        npz = np.load(os.path.join(output_dir, file_name))
        matches = npz['matches']
        valid = matches > -1
        kp1 = npz['keypoints0'][valid]
        kp2 = npz['keypoints1'][matches[valid]]
        return kp1, kp2

    def crop_black(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        coords = cv2.findNonZero(thresh)
        x, y, w, h = cv2.boundingRect(coords)
        return image[y:y+h, x:x+w]

    def stitch_from_numpy_matches(img1, img2, kp1, kp2):
        pts1 = np.float32(kp1).reshape(-1, 1, 2)
        pts2 = np.float32(kp2).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(pts2, pts1, cv2.RANSAC)
        if H is None:
            raise ValueError("Homography failed.")
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        corners = np.float32([[0,0], [0,h2], [w2,h2], [w2,0]]).reshape(-1,1,2)
        warped_corners = cv2.perspectiveTransform(corners, H)
        all_corners = np.concatenate((np.float32([[0,0], [0,h1], [w1,h1], [w1,0]]).reshape(-1,1,2), warped_corners), axis=0)
        [xmin, ymin] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
        [xmax, ymax] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
        translation = [-xmin, -ymin]
        H_translation = np.array([[1, 0, translation[0]],
                                  [0, 1, translation[1]],
                                  [0, 0, 1]])
        result = cv2.warpPerspective(img2, H_translation @ H, (xmax - xmin, ymax - ymin))
        result[translation[1]:h1+translation[1], translation[0]:w1+translation[0]] = img1
        return crop_black(result)

    # Generate .npz filename safely
    img1_base = os.path.splitext(os.path.basename(img1_name))[0]
    img2_base = os.path.splitext(os.path.basename(img2_name))[0]
    match_file = f"{img1_base}_{img2_base}_matches.npz"

    # Run SuperGlue matching
    generate_pair_txt(temp_txt, img1_name, img2_name)
    if not os.path.exists(os.path.join(output_dir, match_file)):
        success = run_superglue(temp_txt)
        if not success:
            raise RuntimeError(f"SuperGlue matching failed between {img1_name} and {img2_name}")

    # Load matches and images
    kp1, kp2 = load_npz_matches(match_file)
    img1 = load_image(img1_name)
    img2 = load_image(img2_name)

    stitched_img = stitch_from_numpy_matches(img1, img2, kp1, kp2)

    if stitched_name is None:
        stitched_name = f"{img1_base}_{img2_base}.jpg"

    stitched_path = os.path.join(input_dir, stitched_name)
    cv2.imwrite(stitched_path, stitched_img)
    return stitched_name


def hierarchical_overlap_stitching(start_idx, end_idx, input_dir, output_dir):
    """
    Iteratively stitch overlapping adjacent image chains:
    Round 0: 1_2, 2_3, 3_4, ...
    Round 1: 1_2_3 (from 1_2 and 2_3), 2_3_4 (from 2_3 and 3_4), ...
    Repeat until 1_2_3_..._n is produced.
    """
    current_images = [f"{i}.jpg" for i in range(start_idx, end_idx + 1)]
    round_num = 0

    while len(current_images) > 1:
        print(f"\n--- Stitching Round {round_num + 1} ---")
        new_images = []
        for i in range(len(current_images) - 1):
            img1 = current_images[i]
            img2 = current_images[i + 1]
            print(f"Stitching {img1} + {img2}")
            try:
                stitched_name = f"stitched_r{round_num}_i{i}.jpg"
                stitched = iterative_superglue_stitch(img1, img2, input_dir, output_dir, stitched_name)
                new_images.append(stitched)
            except Exception as e:
                print(f"❌ Skipping pair {img1} + {img2} due to error: {e}")

        if not new_images:
            raise RuntimeError("All stitching attempts failed. No images to proceed with.")

        current_images = new_images
        round_num += 1

    print(f"\n✅ Final stitched image: {current_images[0]}")
    return current_images[0]


# Call it like this:
hierarchical_overlap_stitching(1, 48, input_dir="InputImages_ref", output_dir="car_panorama2/output/")



--- Stitching Round 1 ---
Stitching 1.jpg + 2.jpg
Stitching 2.jpg + 3.jpg
Stitching 3.jpg + 4.jpg
Stitching 4.jpg + 5.jpg
Stitching 5.jpg + 6.jpg
Stitching 6.jpg + 7.jpg
Stitching 7.jpg + 8.jpg
Stitching 8.jpg + 9.jpg
Stitching 9.jpg + 10.jpg
Stitching 10.jpg + 11.jpg
Stitching 11.jpg + 12.jpg
Stitching 12.jpg + 13.jpg
Stitching 13.jpg + 14.jpg
Stitching 14.jpg + 15.jpg
Stitching 15.jpg + 16.jpg
Stitching 16.jpg + 17.jpg
Stitching 17.jpg + 18.jpg
Stitching 18.jpg + 19.jpg
Stitching 19.jpg + 20.jpg
Stitching 20.jpg + 21.jpg
Stitching 21.jpg + 22.jpg
Stitching 22.jpg + 23.jpg
Stitching 23.jpg + 24.jpg
Stitching 24.jpg + 25.jpg
Stitching 25.jpg + 26.jpg
Stitching 26.jpg + 27.jpg
Stitching 27.jpg + 28.jpg
Stitching 28.jpg + 29.jpg
Stitching 29.jpg + 30.jpg
Stitching 30.jpg + 31.jpg
Stitching 31.jpg + 32.jpg
Stitching 32.jpg + 33.jpg
Stitching 33.jpg + 34.jpg
Stitching 34.jpg + 35.jpg
Stitching 35.jpg + 36.jpg
Stitching 36.jpg + 37.jpg
Stitching 37.jpg + 38.jpg
Stitching 38.jpg + 39.jpg
St

'stitched_r46_i0.jpg'